In [13]:
import arkouda as ak
import arachne as ar
import pandas as pd
import numpy as np
import networkx as nx
import time
import json

In [14]:
# These are just wedges, we do not need subgraph isomorphism for these.
src0 = [    1,     1]
dst0 = [10002, 10003]
connection_type0 = [0, 0]

src1 = [    1,     1, 10003, 10003]
dst1 = [10002, 10003, 30004, 30005]
connection_type1 = [0, 0, 0, 0]

src2 = [    1,     1, 40005]
dst2 = [10002, 10003,     1]
connection_type2 = [0, 0, 1]

src3 = [    1,     1, 40005, 40005, 50008]
dst3 = [10002, 10003, 50008,     1, 10003]
connection_type3 = [0, 0, 0, 1, 1]

src4 = [1, 1, 10003, 10003, 60007]
dst4 = [10002, 10003, 30004, 30005, 1]
connection_type4 = [0, 0, 0, 0, 1]

src5 = [1, 1, 10003, 10003, 60007, 60007, 70010]
dst5 = [10002, 10003, 30004, 30005, 70010, 1, 30005]
connection_type5 = [0, 0, 0, 0, 0, 1, 1]

src6 = [1, 1, 40005, 40005, 80009, 80009, 10003, 90010]
dst6 = [10002, 10003, 50006, 50007, 90010, 90011, 50006, 50007]
connection_type6 = [0, 0, 0, 0, 0, 0, 1, 1]

src7 = [1, 10002, 40005, 60007, 80009]
dst7 = [10002, 20003, 1, 10002, 20003]
connection_type7 = [0, 0, 1, 1, 1]

src00 = [1, 1, 1, 1, 10002]
dst00 = [10002, 10003, 10004, 50006, 70008]
connection_type00 = [0, 0, 0, 1, 1]

src01 = [1, 10002, 20003, 30004, 1, 40005]
dst01 = [10002, 20003, 30004, 40005, 60007, 80009]
connection_type01 = [0, 0, 0, 0, 1, 1]

src02 = [1, 30004, 60007, 90010, 110012, 130014]
dst02 = [10002, 40005, 70008, 1, 30004, 60007]
connection_type02 = [0, 0, 0, 1, 1, 1]

src03 = [1, 1, 1, 50006, 50006, 50006, 120013, 120013, 120013, 10003, 10004]
dst03 = [10002, 10003, 10004, 60007, 60008, 60009, 130014, 130015, 130016, 60007, 130015]
connection_type03 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]

raw_subgraph_data = {
    "0": (src0, dst0, connection_type0),
    "3": (src3, dst3, connection_type3),
    "2": (src2, dst2, connection_type2),
    "7": (src7, dst7, connection_type7),
    "5": (src5, dst5, connection_type5),
    # "4": (src4, dst4, connection_type4),
    # "1": (src1, dst1, connection_type1),
    # "6": (src6, dst6, connection_type6),
    "00": (src00, dst00, connection_type00),
    "01": (src01, dst01, connection_type01),
    "03": (src03, dst03, connection_type03),
    # "02": (src02, dst02, connection_type02)
}

In [15]:
# NOTE: Make sure to change the server name to whatever is applicable in your environment. If running locally, then use only ak.connect().
ak.connect("n116", 5555)

connected to arkouda server tcp://*:5555


In [16]:
# Read in the dataset with pandas.
df = pd.read_csv("/scratch/users/oaa9/arkouda-njit/arachne/data/OL_dataset.csv")
transformed_dataset = ak.DataFrame(df.to_dict(orient='list'))

/tmp/ipykernel_379288/2510305971.py:2: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/scratch/users/oaa9/arkouda-njit/arachne/data/OL_dataset.csv")


In [17]:
# Change to string data to integers.
transformed_dataset["connection_type"] = ak.where(transformed_dataset["connection_type"] == "n", 0, 1)

In [18]:
# Keep only the edge information and "connection_type" attribute.
reduced_dataset = transformed_dataset["src", "dst", "connection_type"]

In [19]:
# Create property graph.
graph = ar.PropGraph()
graph.load_edge_attributes(reduced_dataset, source_column="src", destination_column="dst")
print(f"Graph has {len(graph):_} vertices and {graph.size():_} edges.")

Graph has 667_711 vertices and 962_796 edges.


In [20]:
def vf2_si(g, h):
    isos_as_vertices = ar.subgraph_isomorphism(g, h, 
                                            semantic_check = "and", algorithm_type = "si",
                                            reorder_type = "structural", return_isos_as = "vertices")
    print(f"We found {len(isos_as_vertices[0])/len(h)} monos inside of the graph")

In [21]:
def vf2_si_probability_reordering(g,h):
    isos_as_vertices = ar.subgraph_isomorphism(g, h, 
                                            semantic_check = "and", algorithm_type = "si",
                                            reorder_type = "probability", return_isos_as = "vertices")
    print(f"We found {len(isos_as_vertices[0])/len(h)} monos inside of the graph")

In [22]:
def vf2_ps(g,h):
    isos_as_vertices = ar.subgraph_isomorphism(g, h, 
                                            semantic_check = "and", algorithm_type = "ps", 
                                            reorder_type = None, return_isos_as = "vertices")
    print(f"We found {len(isos_as_vertices[0])/len(h)} monos inside of the graph")

In [23]:
def vf2_ps_structural_reordering(g,h):
    isos_as_vertices = ar.subgraph_isomorphism(g, h, 
                                            semantic_check = "and", algorithm_type = "ps", 
                                            reorder_type = "structural", return_isos_as = "vertices")
    print(f"We found {len(isos_as_vertices[0])/len(h)} monos inside of the graph")

In [24]:
for key,value in raw_subgraph_data.items():
    print(f"Building subgraph{key}...")
    subgraph_dict = {
        "src": value[0],
        "dst": value[1],
        "connection_type": value[2]
    }
    
    subgraph = ar.PropGraph()
    df = ak.DataFrame(subgraph_dict)
    subgraph.load_edge_attributes(df, source_column="src", destination_column="dst")

    print(f"Running VF2-SI on subgraph{key}...")
    vf2_si(graph, subgraph)
    print()

    print(f"Running VF2-SI with probability reordering on subgraph{key}...")
    vf2_si_probability_reordering(graph, subgraph)
    print()

    # print(f"Running VF2-PS on subgraph{key}...")
    # vf2_ps(graph, subgraph)
    # print()

    # print(f"Running VF2-PS with structural reordering on subgraph{key}...")
    # vf2_ps_structural_reordering(graph, subgraph)
    # print()

Building subgraph0...
Running VF2-SI on subgraph0...
We found 696460.0 monos inside of the graph

Running VF2-SI with probability reordering on subgraph0...
We found 696460.0 monos inside of the graph

Building subgraph3...
Running VF2-SI on subgraph3...
We found 5048.0 monos inside of the graph

Running VF2-SI with probability reordering on subgraph3...
We found 5048.0 monos inside of the graph

Building subgraph2...
Running VF2-SI on subgraph2...
We found 191690.0 monos inside of the graph

Running VF2-SI with probability reordering on subgraph2...
We found 191690.0 monos inside of the graph

Building subgraph7...
Running VF2-SI on subgraph7...
We found 91718.0 monos inside of the graph

Running VF2-SI with probability reordering on subgraph7...
We found 91718.0 monos inside of the graph

Building subgraph5...
Running VF2-SI on subgraph5...
We found 2308.0 monos inside of the graph

Running VF2-SI with probability reordering on subgraph5...
We found 2308.0 monos inside of the graph

